In [18]:
import os
import pandas as pd
import numpy as np

In [23]:
df = pd.DataFrame()
for root, _, files in os.walk("raw_data"):
    for filename in files:
        temp_df = pd.read_excel(os.path.join(root, filename), "Sheet1")
        temp_df.drop(columns=[temp_df.columns[0], "Разница в долларах", "Дельта в процентах"], axis=1, inplace=True, errors="ignore")
        df = pd.concat([df, temp_df], axis=0, sort=False)
        
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43169 entries, 0 to 1497
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Дата           22976 non-null  object 
 1   Время          22976 non-null  object 
 2   Цена до        22976 non-null  float64
 3   Цена после     22976 non-null  float64
 4   Текст новости  22976 non-null  object 
 5   Unnamed: 1     5799 non-null   object 
 6   Unnamed: 2     5799 non-null   float64
 7   Unnamed: 3     5799 non-null   float64
 8   Unnamed: 4     5799 non-null   object 
 9   Unnamed: 5     5614 non-null   object 
dtypes: float64(4), object(6)
memory usage: 3.6+ MB


In [24]:
df.head(5)

,Дата,Время,Цена до,Цена после,Текст новости,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,2022-10-10,14:01:25,50.560001,50.445000,Another 'Yuge' Setback For Donald Trump's SPAC...,NaN,NaN,NaN,NaN,NaN
1,2022-10-07,14:02:08,49.410000,49.130001,Twitter Whale Trades For October 07\n,NaN,NaN,NaN,NaN,NaN
2,2022-10-06,15:10:35,50.430000,50.435001,'A Suicide Bomb': NBC Reporter Details How Mus...,NaN,NaN,NaN,NaN,NaN
3,2022-10-06,13:35:35,50.720001,50.430000,Selling Tesla Shares To Buy Twitter Is 'Giving...,NaN,NaN,NaN,NaN,NaN
4,2022-10-06,11:26:44,50.950100,51.040001,New Book Shows Donald Trump Treated White Hous...,NaN,NaN,NaN,NaN,NaN


In [27]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head()

,Дата,Время,Цена до,Цена после,Текст новости
0,2022-10-10,14:01:25,50.560001,50.445000,Another 'Yuge' Setback For Donald Trump's SPAC...
1,2022-10-07,14:02:08,49.410000,49.130001,Twitter Whale Trades For October 07\n
2,2022-10-06,15:10:35,50.430000,50.435001,'A Suicide Bomb': NBC Reporter Details How Mus...
3,2022-10-06,13:35:35,50.720001,50.430000,Selling Tesla Shares To Buy Twitter Is 'Giving...
4,2022-10-06,11:26:44,50.950100,51.040001,New Book Shows Donald Trump Treated White Hous...


In [39]:
temp_df = df.drop_duplicates(subset=["Текст новости"])
temp_df = temp_df.rename(columns={"Цена до": "price_before", "Цена после": "price_after", "Дата": "date", "Время": "Time", "Текст новости": "news_text"})

temp_df["absolute_price_difference"] = temp_df["price_after"] - temp_df["price_before"]
temp_df["news_text"] = temp_df["news_text"]
temp_df["label"] = np.where(temp_df["absolute_price_difference"] > 0, 1, 0)
temp_df = temp_df[["news_text", "label"]].copy()
df = temp_df
df.head()

,news_text,label
0,Another 'Yuge' Setback For Donald Trump's SPAC...,0
1,Twitter Whale Trades For October 07\n,0
2,'A Suicide Bomb': NBC Reporter Details How Mus...,1
3,Selling Tesla Shares To Buy Twitter Is 'Giving...,0
4,New Book Shows Donald Trump Treated White Hous...,1


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5535 entries, 0 to 667
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   news_text  5534 non-null   object
 1   label      5535 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 129.7+ KB


In [41]:
df.head()

,news_text,label
0,Another 'Yuge' Setback For Donald Trump's SPAC...,0
1,Twitter Whale Trades For October 07\n,0
2,'A Suicide Bomb': NBC Reporter Details How Mus...,1
3,Selling Tesla Shares To Buy Twitter Is 'Giving...,0
4,New Book Shows Donald Trump Treated White Hous...,1


In [43]:
# make dir for whole data

import os

if not os.path.exists("data"):
    os.mkdir("data")

df.to_csv("data/preprocessed_data.csv", sep=";")